In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
label_encoding={label:num for num,label in enumerate(words)}
